In [1]:
# Import libraries - REQUIRES pip version 9.0.3
import pandas
import os
from os.path import join
import sys
import scipy.stats
import numpy
import math
import pickle
import copy
import time
import warnings
#warnings.filterwarnings("ignore", category=UserWarning) # ignore errors in some chemical formulas

# Using Cobrapy 0.13.0
import cobra
import cobra.test
from cobra.io import write_sbml_model
from cobra.flux_analysis import  flux_variability_analysis
from cobra.flux_analysis import gapfill
from cobra.flux_analysis.reaction import assess_component
from cobra.manipulation.delete import *
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.medium import find_boundary_types
#from cobra.flux_analysis.sampling import OptGPSampler

# Driven worflow for transcript integration
#from driven.flux_analysis.transcriptomics import gimme
#from driven.flux_analysis.transcriptomics import imat
#from driven.data_sets.expression_profile import ExpressionProfile
#from corda import CORDA

# Confidence levels in each annotation
import probanno

# To generate new stoichiometry for biomass function
import BOFdat

# Using Gurobi solver instead of GLPK
import gurobipy
from optlang import gurobi_interface

# Import scripted python functions
sys.path.append('code/python/')
from read_excel import read_excel

# Estabish handler for logger
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

# Verbose exception printing
%xmode

Exception reporting mode: Verbose


In [2]:
# Actually prune all unused metabolites and reactions (innate function does not work)
def removeUnused(model):
    removed_cpd = set()
    removed_rxn = set()
    unused_current_cpd = 1
    unused_current_rxn = 1
    
    while unused_current_cpd != 0 or unused_current_rxn != 0:
        unused_cpd = prune_unused_metabolites(model)
        removed_cpd |= set(unused_cpd)
        unused_rxn = prune_unused_reactions(model)
        removed_rxn |= set(unused_rxn)
        
        unused_current_cpd = len(unused_cpd)
        unused_current_rxn = len(unused_rxn)
    
    print('Pruned ' + str(len(removed_cpd)) + ' metabolites from model')
    print('Pruned ' + str(len(removed_rxn)) + ' reactions from model')
        
    return(list(removed_cpd), list(removed_rxn))


In [2]:
#mackinac.list_workspace_objects('/chenry/public/modelsupport/templates', print_output=True)
#universal = mackinac.create_universal_model('/chenry/public/modelsupport/templates/GramPosModelTemplate')
#cobra.io.save_json_model(universal, "../data/GramPosUni.json")

In [10]:
universal = cobra.io.load_json_model('data/GramPosUni.json')

In [11]:
universal

Name,GramPositive.modeltemplate
Memory address,0x07f72a053ee10
Number of metabolites,6884
Number of reactions,8657
Objective expression,0
Compartments,"c, e"


In [12]:
# Remove ModelSEED annotated reactions containing oxygen
universal.metabolites.cpd00007_c.remove_from_model(destructive=True)
universal.metabolites.cpd00007_e.remove_from_model(destructive=True)

In [13]:
# Remove remnants of old biomass reactions
biomass_ids = ['rxn13783_c', 'rxn13784_c', 'rxn13782_c', 'bio1', 'SK_cpd11416_c']
removed = 0
for rxn in biomass_ids: 
    try:
        universal.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)
        removed += 1
    except:
        pass
print('Removed: ' + str(removed))

Removed: 3


In [14]:
# Remove reactions with unbalanced formulas
removed = 0
for rxn in universal.reactions:
    if len(list(rxn.check_mass_balance())) > 0: 
        rxn.remove_from_model(remove_orphans=True)
        removed += 1
print('Removed ' + str(removed) + ' imbalanced reactions')

Removed 156 imbalanced reactions


In [15]:
Rs = 0
for cpd in universal.metabolites:
    if 'R' in str(cpd.formula):
        cpd.remove_from_model(destructive=True)
        Rs += 1
        
print('Reactions with R-groups: ' + str(Rs))

Reactions with R-groups: 732


In [16]:
# removed unused metabolites and reactions
unused_cpd, unused_rxn = removeUnused(universal)

Pruned 546 metabolites from model
Pruned 8 reactions from model


In [17]:
# Reformat metabolites
for cpd in universal.metabolites:
        
    # Improve metabolite annotation
    cpd.notes = {'annotation': 'ModelSEED'}
    if cpd.compartment == 'c':
        cpd.compartment = 'cytosol'
    elif cpd.compartment == 'e':
        cpd.compartment = 'extracellular'

In [18]:
# Reformat reactions
for rxn in universal.reactions:
    
    # Add additional annotation
    rxn.notes = {'citations': [], 'confidence': 0.0, 'type': 'metabolic', 'annotation': 'ModelSEED'}
    
    # Label general reaction type
    substrates = set([x.id.split('_')[-1] for x in rxn.products] + [y.id.split('_')[-1] for y in rxn.reactants])
    if rxn in universal.boundary:
        rxn.notes['type'] = 'exchange'
    elif 'e' in substrates and 'c' in substrates:
        rxn.notes['type'] = 'transport'
        

In [19]:
universal.name = 'Universal Gram-positive'
universal.id = 'Universal Gram-positive'

In [20]:
universal

Name,Universal Gram-positive
Memory address,0x07f72a053ee10
Number of metabolites,5554
Number of reactions,6647
Objective expression,0
Compartments,"cytosol, extracellular"


In [21]:
cobra.io.save_json_model(universal, 'data/universal.json')